In [3]:
import wrangle as w
import model as m
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
import nltk
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [4]:
train = pd.read_parquet('train.parquet')
validate = pd.read_parquet('validate.parquet')
test = pd.read_parquet('test.parquet')

In [5]:
sm_train1 = int(round(len(train[train.company_response_to_consumer=='Closed with explanation'])*.2,0))
sm_train2 = int(round(len(train[train.company_response_to_consumer=='Closed with non-monetary relief'])*.2,0))
sm_train3 = int(round(len(train[train.company_response_to_consumer=='Closed with monetary relief'])*.2,0))
sm_train4 = int(round(len(train[train.company_response_to_consumer=='Untimely response'])*.2,0))
sm_train5 = int(round(len(train[train.company_response_to_consumer=='Closed'])*.2,0))
sm_val1 = int(round(len(validate[validate.company_response_to_consumer=='Closed with explanation'])*.2,0))
sm_val2 = int(round(len(validate[validate.company_response_to_consumer=='Closed with non-monetary relief'])*.2,0))
sm_val3 = int(round(len(validate[validate.company_response_to_consumer=='Closed with monetary relief'])*.2,0))
sm_val4 = int(round(len(validate[validate.company_response_to_consumer=='Untimely response'])*.2,0))
sm_val5 = int(round(len(validate[validate.company_response_to_consumer=='Closed'])*.2,0))
sm_test1 = int(round(len(test[test.company_response_to_consumer=='Closed with explanation'])*.2,0))
sm_test2 = int(round(len(test[test.company_response_to_consumer=='Closed with non-monetary relief'])*.2,0))
sm_test3 = int(round(len(test[test.company_response_to_consumer=='Closed with monetary relief'])*.2,0))
sm_test4 = int(round(len(test[test.company_response_to_consumer=='Untimely response'])*.2,0))
sm_test5 = int(round(len(test[test.company_response_to_consumer=='Closed'])*.2,0))

small_train1 = train[train.company_response_to_consumer=='Closed with explanation'].sample(sm_train1,random_state=123)
small_train2 = train[train.company_response_to_consumer=='Closed with non-monetary relief'].sample(sm_train2,random_state=123)
small_train3 = train[train.company_response_to_consumer=='Closed with monetary relief'].sample(sm_train3,random_state=123)
small_train4 = train[train.company_response_to_consumer=='Untimely response'].sample(sm_train4,random_state=123)
small_train5 = train[train.company_response_to_consumer=='Closed'].sample(sm_train5,random_state=123)
small_val1 = validate[validate.company_response_to_consumer=='Closed with explanation'].sample(sm_val1,random_state=123)
small_val2 = validate[validate.company_response_to_consumer=='Closed with non-monetary relief'].sample(sm_val2,random_state=123)
small_val3 = validate[validate.company_response_to_consumer=='Closed with monetary relief'].sample(sm_val3,random_state=123)
small_val4 = validate[validate.company_response_to_consumer=='Untimely response'].sample(sm_val4,random_state=123)
small_val5 = validate[validate.company_response_to_consumer=='Closed'].sample(sm_val5,random_state=123)
small_test1 = test[test.company_response_to_consumer=='Closed with explanation'].sample(sm_test1,random_state=123)
small_test2 = test[test.company_response_to_consumer=='Closed with non-monetary relief'].sample(sm_test2,random_state=123)
small_test3 = test[test.company_response_to_consumer=='Closed with monetary relief'].sample(sm_test3,random_state=123)
small_test4 = test[test.company_response_to_consumer=='Untimely response'].sample(sm_test4,random_state=123)
small_test5 = test[test.company_response_to_consumer=='Closed'].sample(sm_test5,random_state=123)

small_train = pd.concat([small_train1,small_train2,small_train3,small_train4,small_train5])
small_val = pd.concat([small_val1,small_val2,small_val3,small_val4,small_val5])
small_test = pd.concat([small_test1,small_test2,small_test3,small_test4,small_test5])

X_train = m.encode(small_train)
X_train = X_train.drop(columns=['date_received','company_response_to_consumer','clean','state','company_name','tags','product_bins'])
y_train = small_train['company_response_to_consumer']
X_val = m.encode(small_val)
X_val = X_val.drop(columns=['date_received','company_response_to_consumer','clean','state','company_name','tags','product_bins'])
y_val = small_val['company_response_to_consumer']
X_test = m.encode(small_test)
X_test = X_test.drop(columns=['date_received','company_response_to_consumer','clean','state','company_name','tags','product_bins'])
y_test = small_test['company_response_to_consumer']
X_train.head()

,lemon,Older American,"Older American, Servicemember",Servicemember,credit_card,credit_report,debt_collection,loans,money_service,mortgage
48045,trying contact grain technology inc debt owed ...,0,0,0,0,0,1,0,0,0
1164880,bought large rug start vacation called filed d...,0,0,0,1,0,0,0,0,0
463976,original complaint went car dealership ohio lo...,0,0,0,0,1,0,0,0,0
490359,synopsis loan demand estate loan originated an...,0,0,0,0,0,0,0,0,1
316427,may concern writing dispute fraudulent charge ...,0,0,0,0,1,0,0,0,0


In [6]:
X_train_cv,X_val_cv,X_test_cv = m.make_cv(X_train, X_val, X_test)
X_train_tf,X_val_tf,X_test_tf = m.make_tfidf(X_train, X_val, X_test)

In [ ]:
encoded_train = X_train.iloc[:,1:]
encoded_val = X_val.iloc[:,1:]
encoded_test = X_test.iloc[:,1:]

X_train_cve = encoded_train.merge(X_train_cv,left_index=True, right_index=True)
X_val_cve = encoded_val.merge(X_val_cv,left_index=True, right_index=True)
X_test_cve = encoded_test.merge(X_test_cv,left_index=True, right_index=True)

X_train_tfe = encoded_train.merge(X_train_tf,left_index=True, right_index=True)
X_val_tfe = encoded_val.merge(X_val_tf,left_index=True, right_index=True)
X_test_tfe = encoded_test.merge(X_test_tf,left_index=True, right_index=True)

In [ ]:
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

In [ ]:
hyperparams = [
    {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 100},
    {'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 200}]

In [ ]:
def xgboost_hyperparam_search(X_train, y_train, X_val, y_val, hyperparams):
    results = []
    
    for params in hyperparams:
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train)
        
        train_pred = model.predict(X_train)
        train_acc = accuracy_score(y_train, train_pred)
        
        val_pred = model.predict(X_val)
        val_acc = accuracy_score(y_val, val_pred)
        
        results.append({'Parameters': params, 'Train Accuracy': train_acc, 'Validation Accuracy': val_acc})
    
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values(by='Validation Accuracy', ascending=False).head(10)
    
    # Plotting the top 10 models
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(df_results) + 1), df_results['Train Accuracy'], label='Train Accuracy')
    plt.plot(range(1, len(df_results) + 1), df_results['Validation Accuracy'], label='Validation Accuracy')
    plt.xlabel('Model')
    plt.ylabel('Accuracy')
    plt.title('Top 10 Models - Train vs Validation Accuracy')
    plt.legend()
    plt.xticks(range(1, len(df_results) + 1))
    plt.show()
    
    return df_results

In [ ]:
xgboost_hyperparam_search(X_train_cve, y_train_encoded, X_val_cve, y_val_encoded, hyperparams)